In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv ("/content/drive/My Drive/DAJUNG_comments.csv")

In [ ]:
df.head()

,comments,name,time,likes,topic,label
0,공대생이셨다니 멋져요😍 다음 라이브는 꼭 시간 맞춰올게요 ㅋㅋ 아숩지뭐에요 ㅜㅜ,Hobby choi game-log,2021-03-09T15:01:15Z,0.0,"0,1",1
1,언제 공지했을까요? 함께 하고 싶었어요 !!아쉽 <br>항상 영상 보면 저도 모르...,Čkd buds,2021-03-06T05:25:18Z,2.0,2,1
2,앗 라이브 보려했는데 아쉬워요 🥺 담에도 시간 괜찮으실때 해주세요!! 👏🏻,또*강누나,2021-03-05T05:58:44Z,3.0,"0,1",1
3,다정님 랜선집들이는 언제나봐도 힐링되고 재밋는거같아요! 집안곳곳 다정님이 고민하시고...,백도,2021-03-07T04:23:30Z,1.0,0,1
4,"상부장 위쪽은 실크벽지로 마감되어있고, 욕실 바닥은 흰색과 하늘색 타일을 섞어서 사...",다정하게DAJUNG,2020-10-25T11:19:10Z,0.0,1,0


In [ ]:
len(df)

1298

In [ ]:
df.isnull().sum()

comments    0
name        0
time        0
likes       0
topic       0
label       0
dtype: int64

In [ ]:
df = df.dropna()
len(df)

2891

**토픽별 긍/부정 점수 계산**

In [ ]:
df['likes'] = df['likes'].astype(float).astype(int)
df.head()

,comments,name,time,likes,topic,label
0,공대생이셨다니 멋져요😍 다음 라이브는 꼭 시간 맞춰올게요 ㅋㅋ 아숩지뭐에요 ㅜㅜ,Hobby choi game-log,2021-03-09T15:01:15Z,0,"0,1",1
1,언제 공지했을까요? 함께 하고 싶었어요 !!아쉽 <br>항상 영상 보면 저도 모르...,Čkd buds,2021-03-06T05:25:18Z,2,2,1
2,앗 라이브 보려했는데 아쉬워요 🥺 담에도 시간 괜찮으실때 해주세요!! 👏🏻,또*강누나,2021-03-05T05:58:44Z,3,"0,1",1
3,다정님 랜선집들이는 언제나봐도 힐링되고 재밋는거같아요! 집안곳곳 다정님이 고민하시고...,백도,2021-03-07T04:23:30Z,1,0,1
4,"상부장 위쪽은 실크벽지로 마감되어있고, 욕실 바닥은 흰색과 하늘색 타일을 섞어서 사...",다정하게DAJUNG,2020-10-25T11:19:10Z,0,1,0


In [ ]:
topic_n = [0]*4
label_1 = [0]*4

In [ ]:
for index, row in df.iterrows():
    topic_n[row['가장 비중이 높은 토픽']] += 1
    if row['label'] == 1:
      label_1[row['가장 비중이 높은 토픽']] += 1

In [ ]:
for i in range(4):
  pos = label_1[i]/topic_n[i]*100
  neg = 100-pos
  print("Topic {0} : 긍정 {1:.2f} %, 부정 {2:.2f} %". format(i, pos, neg))

Topic 0 : 긍정 50.13 %, 부정 49.87 %
Topic 1 : 긍정 46.55 %, 부정 53.45 %
Topic 2 : 긍정 47.77 %, 부정 52.23 %
Topic 3 : 긍정 51.92 %, 부정 48.08 %


**각 토픽별 요약하기**

In [ ]:
pip install gensim newspaper3k

     |████████████████████████████████| 215kB 5.4MB/s 
     |████████████████████████████████| 81kB 5.3MB/s 
     |████████████████████████████████| 92kB 5.8MB/s 
     |████████████████████████████████| 7.4MB 8.5MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp37-none-any.whl size=13553 sha256=af3b07b34476fb212cdfeb7d272b278307e4548dfb60d58f197c24d2e113ec02
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp37-none-any.whl size=7398404 sha256=fece7560c65bd5d9d9f63c75941ad20a58f7b2233bf5dfd64ce31feea3c652fe
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp37-none-any.whl size=3358 sha256=706967ffbe2e936322906546a2d650d43c337f30706f7e698a668a7318a67884
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c75

In [ ]:
import re

# html 태그 없애기
cleanr =re.compile('<.*?>')

#이모티콘 제거
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

#분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
# han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d]')
# 한글 외 영어, 숫자 등 제거
han = re.compile(r'[^가-힣 ]')

In [ ]:
# 전처리
cleantext = []
for i in df['comments']:
  text = re.sub(cleanr,"",i)
  text = re.sub(emoji_pattern,"",text)
  text = re.sub(han,"",text)
  cleantext.append(text)

In [ ]:
df['cleantext'] = cleantext
df.head()

,comments,name,time,likes,topic,label,cleantext
0,공대생이셨다니 멋져요😍 다음 라이브는 꼭 시간 맞춰올게요 ㅋㅋ 아숩지뭐에요 ㅜㅜ,Hobby choi game-log,2021-03-09T15:01:15Z,0,"0,1",1,공대생이셨다니 멋져요 다음 라이브는 꼭 시간 맞춰올게요 아숩지뭐에요
1,언제 공지했을까요? 함께 하고 싶었어요 !!아쉽 <br>항상 영상 보면 저도 모르...,Čkd buds,2021-03-06T05:25:18Z,2,2,1,언제 공지했을까요 함께 하고 싶었어요 아쉽 항상 영상 보면 저도 모르게 부지런히 ...
2,앗 라이브 보려했는데 아쉬워요 🥺 담에도 시간 괜찮으실때 해주세요!! 👏🏻,또*강누나,2021-03-05T05:58:44Z,3,"0,1",1,앗 라이브 보려했는데 아쉬워요 담에도 시간 괜찮으실때 해주세요
3,다정님 랜선집들이는 언제나봐도 힐링되고 재밋는거같아요! 집안곳곳 다정님이 고민하시고...,백도,2021-03-07T04:23:30Z,1,0,1,다정님 랜선집들이는 언제나봐도 힐링되고 재밋는거같아요 집안곳곳 다정님이 고민하시고 ...
4,"상부장 위쪽은 실크벽지로 마감되어있고, 욕실 바닥은 흰색과 하늘색 타일을 섞어서 사...",다정하게DAJUNG,2020-10-25T11:19:10Z,0,1,0,상부장 위쪽은 실크벽지로 마감되어있고 욕실 바닥은 흰색과 하늘색 타일을 섞어서 사용...


In [ ]:
# [["Topic0 부정","Topic0 긍정"], ["Topic1 부정","Topic1 긍정"], ["Topic2 부정","Topic2 긍정"], ["Topic3 부정","Topic3 긍정"]]
TopicText = [["" for col in range(2)] for row in range(4)]

for i in range(len(df)):
  for j in range(4):
    if str(j) in df['topic'][i]:
      if df['label'][i] == 0: # 부정 댓글
        TopicText[j][0] += cleantext[i]+". "
      else:         # 긍정 댓글
        TopicText[j][1] += cleantext[i]+". "

In [ ]:
TopicText

[[' 고민부부 집 꾸미기  몰아보기  제품 정보는 더보기에 있어용 블로그도 빨리 작성 해 볼게요                 빈 벽 채우기  선반에 가림막 커튼 달기  커튼 교체  베란다 팬트리 정리  선반 정리  커튼 다리고 마무리. 인가구에 컵 그정도 있으신건 양반이신건데 저희집은 인가구 같은 인가구인데 가지고 계신것기준으로 하면 최소 배정도  저희 엄마가 그릇 욕심이 장난이 아니거든요 저희집도 이제 슬슬 정리하고 해야 하는데정리할곳을 메모하다보니 한숨이 아휴거기다 옷장도 정리하고 해야 하는데이번주안에 다 할수 있을까요 하아. 저도 주방베란다 대청소해야하는데 자꾸모른척중   이렇게 영상으로 대리만족하고 갈께용. 빨래 건조는 건조기가 주방에 있고 옷은 베란다에 널어놓아요  주방정리 열심히 찍어 보겠슴다 . 박종신시장생활 저희는 주로 집에서 해먹고 잘 안사먹고 있어요  제가 아주 어릴 때 부터 집에서 엄마를 돕느라 집안일을 많이 했어서 밥하고 살림하는게 익숙한 것 같아요 . 저 궁금한게 있어요 요리할때 물도 흘리고 음식물도 한번씩 흘리는데 그럴때마다 걸레로 닦거든요 근데 닦고 손빨래를 하는데 너무 팔이 아파요 하루에 번을 빨아요 청소기돌리고 바닥 닦는 걸레도 있는데 그건 따로 쓰거든요 세탁기로 하기에는 물이 아깝네요 이럴땐 어떻하죠. 우와 부엌하나 한시간동안 청소하고 현타와서 보는데 정말 부지런하시네요 전 아직 화장실과 안방이 남았습니다 일주일에 세번이라니. 맞벌이시면 평일 청소는 퇴근하고 저녁드시고 밤에 하시나요 청소시간 총 어느정도 걸리시는지도 궁금해요침구는 그냥 정리만 하고 침구털기나 침구청소기돌리는건 안하시나용. 한편한편 볼때마다 시간순삭이에요.    오늘 올린 영상 분쯤에 베란다가 어떻게 변했는지 슬쩍 올려 두었어요  알로카시아는 물에 담궈 두었던건 결국 물러서 살리지 못했고 화분에 심어둔건 파보니까 뿌리를 내리기 시작했더라구요 . 제가 정말 무심한건지 저는 식물키우기가 너무 어렵더라구요 가만히 있는 놈이니 나를 불러주지않아서 반려동물보다 반려 식물

In [ ]:
from gensim.summarization.summarizer import summarize

In [ ]:
TopicText[0][0]

' 고민부부 집 꾸미기  몰아보기  제품 정보는 더보기에 있어용 블로그도 빨리 작성 해 볼게요                 빈 벽 채우기  선반에 가림막 커튼 달기  커튼 교체  베란다 팬트리 정리  선반 정리  커튼 다리고 마무리. 인가구에 컵 그정도 있으신건 양반이신건데 저희집은 인가구 같은 인가구인데 가지고 계신것기준으로 하면 최소 배정도  저희 엄마가 그릇 욕심이 장난이 아니거든요 저희집도 이제 슬슬 정리하고 해야 하는데정리할곳을 메모하다보니 한숨이 아휴거기다 옷장도 정리하고 해야 하는데이번주안에 다 할수 있을까요 하아. 저도 주방베란다 대청소해야하는데 자꾸모른척중   이렇게 영상으로 대리만족하고 갈께용. 빨래 건조는 건조기가 주방에 있고 옷은 베란다에 널어놓아요  주방정리 열심히 찍어 보겠슴다 . 박종신시장생활 저희는 주로 집에서 해먹고 잘 안사먹고 있어요  제가 아주 어릴 때 부터 집에서 엄마를 돕느라 집안일을 많이 했어서 밥하고 살림하는게 익숙한 것 같아요 . 저 궁금한게 있어요 요리할때 물도 흘리고 음식물도 한번씩 흘리는데 그럴때마다 걸레로 닦거든요 근데 닦고 손빨래를 하는데 너무 팔이 아파요 하루에 번을 빨아요 청소기돌리고 바닥 닦는 걸레도 있는데 그건 따로 쓰거든요 세탁기로 하기에는 물이 아깝네요 이럴땐 어떻하죠. 우와 부엌하나 한시간동안 청소하고 현타와서 보는데 정말 부지런하시네요 전 아직 화장실과 안방이 남았습니다 일주일에 세번이라니. 맞벌이시면 평일 청소는 퇴근하고 저녁드시고 밤에 하시나요 청소시간 총 어느정도 걸리시는지도 궁금해요침구는 그냥 정리만 하고 침구털기나 침구청소기돌리는건 안하시나용. 한편한편 볼때마다 시간순삭이에요.    오늘 올린 영상 분쯤에 베란다가 어떻게 변했는지 슬쩍 올려 두었어요  알로카시아는 물에 담궈 두었던건 결국 물러서 살리지 못했고 화분에 심어둔건 파보니까 뿌리를 내리기 시작했더라구요 . 제가 정말 무심한건지 저는 식물키우기가 너무 어렵더라구요 가만히 있는 놈이니 나를 불러주지않아서 반려동물보다 반려 식물 돌

In [ ]:
# Topic0 부정 댓글 요약
print(summarize(TopicText[0][0], word_count=20))

안녕하세요요즘 이사하면서 인테리어 검색하다 우연히 봤는데 구독해서 보게 되네요  그래서 그런가 집을 점점 비슷하게 하려고 하는거 같은 혹 베란다 창문에 화분 걸어두신 막대기 걸이 어디서 사야하는지 정보 좀 알려주세요.


In [ ]:
TopicText[0][1]

'공대생이셨다니 멋져요 다음 라이브는 꼭 시간 맞춰올게요  아숩지뭐에요 . 앗 라이브 보려했는데 아쉬워요  담에도 시간 괜찮으실때 해주세요 . 다정님 랜선집들이는 언제나봐도 힐링되고 재밋는거같아요 집안곳곳 다정님이 고민하시고 손길하나하나 안닿은곳없이 그저 예쁘다라는말만 연발 내뱉고있네요  전체적인 인테리어가 좋은데 개인적으로 화장실이랑 베란다 너무 좋아욬 그리고 주방한켠에 자리잡은 다이닝룸도 다양한 공간으로 활용할수있는거같아서 재밋네요.  식물이름 알수 있을까용. 우와 진짜 너무너무 이쁘네요 베란다 타일 하늘색으로 하니까 진짜 시원해보이고 탁 트여보여요 짱이다. 오앙 뜨개와 자수를 좋아하시는군요 반가와요 집도 넘 이뻐요. 집이 너무 예뻐요 감각적이고 포근해보여요 영상 보는 동안 잠시 다른 세계에 온 것 같았어요 좋은 영상 감사합니다 그리구 혹시 베란다에 걸어놓으신    액자 구매처 알 수 있을까요 너무 좋아하는 그림이라 방에 걸어두고싶어요. 베란다펜트리너무 예쁘네요펜트리책장 정보 좀 알고싶어요. 요즘 고민부부로 하루를 시작하고 마무리 합니다 우연히 다이아 에서 만난후 팬이 되었어요 꽃무늬 가림커튼도 질러버렸다는 덕분에  눈이 높아집니다 . 우와  베란다 팬드리 선반은 어느 제품인지 알려주실 수 있나요  너무 깔끔해보여요 . 베란다 선반은 이케아에서 구입했어요 . 다정하게 늦은 시간인데 빠른 답 참 감사합니다 성능기능도 좋은가요. 베란다 커튼 어디서 구입하셨는지 궁금해요. 저정도면 깨끗한건데 울집은 발디딜려면 물건을 하나씩 치우고 다용도실을 다녀야 한답니다 빨래건조는 어디다 하세요베란다쪽을 예쁘게 꾸며 놓으셔서 항상 궁금했다능참 주방정리 보고싶어요. 하얗고 깨끗해서 좋긴 한데 먼지가 많이 들어오진 않나요베란다도 환기가 중요해서 살짝 창문을 열어두니 먼지가. 아 저는 엄마살림을 보고 배워서 저렇게 하는거에요  각자 청소 방법이 다를 것 같아요 . 다정하게 답변   넘  감사드려요 넘  예쁜  화장실이라  리모델링  할때하고픈  스타일이에요샤워부스보다   실용적으로

In [ ]:
# Topic0 긍정 댓글 요약
print(summarize(TopicText[0][1], word_count=200))

몰디브이탈리아오사카파리방콕 다정하게님 영상은 항상 뭔가 느낌있고 특히 갬성이 있어서 보면은 꼭 노근하고 힐링하는 느낌라서 막 기분이 좋아져요 그래서 뭔가 여유가 필요할때 꼭 다정하게님 영상을 보게 되요  그리고 방 꾸미기와 관련한 영상을 많이 올려주셔서 방 꾸미기에 관심이 많은 분들께 도움이 많이 될 거 같아요 저도 방을 예쁘게 제 취향에 맞게 꾸미는 것에 대해 관심이 많아서 기분에 따라 방을 자주 꾸며요 방 꾸미기 영상을 많이 찾아보는 편인데 정말 도움 많이 됐어요 감사해요 이 고마운 마음을 제가 가장 최애하는 딸기를 마음으로 보내드릴게요 아직까지는 사진으로 벽을 꾸민적이 없는데 이 영상을 보니까 갑자기 제가 지금까지 소중하게 생각하는 사진미국에 갔었던 사진가족과 함께 놀이공원에 간 사진 제주도에 놀러가서 사촌과 찍었던사진 소품샵에서 찍었던 사진 을 프린트해서 허전한 벽을 채우면 예쁜고 거기다 저의 추억까지 담는 거 같아서 정말 좋은 생각인 거 같아요 사실 예전에 한 약속때문에 이번 년에 친구들과 함께  박일로 여행을 가기로 했는데 코로나때문에 아쉽게 여행 취소 됐어요 밖에도 못 나가고 친구들 못 만난지도 오래되서 요즘 갑자기 자주 기분이 우울했거든요  그런데 다정하게님 영상들을 보니까 너무 분위기가 좋아서 그런지 기분이 좋아지는 거 같아요  코로나가 생긴뒤로 저의 버킷리스트에 코로나가 종식되면 바로 여행을 가서 예쁜 옷 입고 맛있는 거 먹는게 추가가 되었는데 예쁜 옷을 입고 사진찍으면 그 사진을 인스탁스 쉐어 로 뽑아서 제 방 책상  위 벽쪽에 잘 보이게 붙인 다음에 그 추억을 간직하고 싶어요  감사하고 항상 행복한 일들만 가득하시길 바래요.
몰디브이탈리아오사카파리방콕 다정하게님 영상은 항상 뭔가 느낌있고 특히 갬성이 있어서 보면은 꼭 노근하고 힐링하는 느낌라서 막 기분이 좋아져요 그래서 뭔가 여유가 필요할때 꼭 다정하게님 영상을 보게 되요  그리고 방 꾸미기와 관련한 영상을 많이 올려주셔서 방 꾸미기에 관심이 많은 분들께 도움이 많이 될 거 같아요 저도 

In [ ]:
# Topic1 부정 댓글 요약
print(summarize(TopicText[1][0], word_count=10))

저는 좀 춥게 살아요  도로 유지하고 난로 틀고 전기장판 틀고 있어요 가습기는 다이슨꺼 사용하고 있는데 매일매일 틀어놓고 있어용 .


In [ ]:
# Topic1 긍정 댓글 요약
print(summarize(TopicText[1][1], word_count=120))

몰디브이탈리아오사카파리방콕 다정하게님 영상은 항상 뭔가 느낌있고 특히 갬성이 있어서 보면은 꼭 노근하고 힐링하는 느낌라서 막 기분이 좋아져요 그래서 뭔가 여유가 필요할때 꼭 다정하게님 영상을 보게 되요  그리고 방 꾸미기와 관련한 영상을 많이 올려주셔서 방 꾸미기에 관심이 많은 분들께 도움이 많이 될 거 같아요 저도 방을 예쁘게 제 취향에 맞게 꾸미는 것에 대해 관심이 많아서 기분에 따라 방을 자주 꾸며요 방 꾸미기 영상을 많이 찾아보는 편인데 정말 도움 많이 됐어요 감사해요 이 고마운 마음을 제가 가장 최애하는 딸기를 마음으로 보내드릴게요 아직까지는 사진으로 벽을 꾸민적이 없는데 이 영상을 보니까 갑자기 제가 지금까지 소중하게 생각하는 사진미국에 갔었던 사진가족과 함께 놀이공원에 간 사진 제주도에 놀러가서 사촌과 찍었던사진 소품샵에서 찍었던 사진 을 프린트해서 허전한 벽을 채우면 예쁜고 거기다 저의 추억까지 담는 거 같아서 정말 좋은 생각인 거 같아요 사실 예전에 한 약속때문에 이번 년에 친구들과 함께  박일로 여행을 가기로 했는데 코로나때문에 아쉽게 여행 취소 됐어요 밖에도 못 나가고 친구들 못 만난지도 오래되서 요즘 갑자기 자주 기분이 우울했거든요  그런데 다정하게님 영상들을 보니까 너무 분위기가 좋아서 그런지 기분이 좋아지는 거 같아요  코로나가 생긴뒤로 저의 버킷리스트에 코로나가 종식되면 바로 여행을 가서 예쁜 옷 입고 맛있는 거 먹는게 추가가 되었는데 예쁜 옷을 입고 사진찍으면 그 사진을 인스탁스 쉐어 로 뽑아서 제 방 책상  위 벽쪽에 잘 보이게 붙인 다음에 그 추억을 간직하고 싶어요  감사하고 항상 행복한 일들만 가득하시길 바래요.


In [ ]:
# Topic2 부정 댓글 요약
print(summarize(TopicText[2][0], word_count=10))

냉장고 정리도 해야하는데 엄두가 안나서 미루고 있어요  늘 감사합니다 .


In [ ]:
# Topic2 긍정 댓글 요약
print(summarize(TopicText[2][1], word_count=10))

요즘 슬슬 다시 일을 시작하고 있는데 좋아하는 일들을 하니까 매사 감사하고 즐거운 것 같아요  감사합니다 저는 용인에 살고 있어요 .


In [ ]:
# Topic3 부정 댓글 요약
print(summarize(TopicText[3][0], word_count=10))

웜그레이테일 제품인데 커튼은 아니고 그냥 패브릭이에요 .
커튼 어디서 구매하셨는지 알고싶어요 .


In [ ]:
# Topic3 긍정 댓글 요약
print(summarize(TopicText[3][1], word_count=20))

재미있게 봐 주셔서 감사합니다  커튼 실물이 더 예뻐요 튀니까 한폭 정도만 포인트로 넣는거 좋은 것 같아요  라탄 휴지케이스는 블랑 드 엔젤에서 구입했어요 .
